### <font color='black'>Q1. (Multinomail Classification)</font>

(a) Randomly shuffle each dataset and use 80% of data for training and 20% for testing. And repeat 10 times and report the average accuracy for each class.  

In [2]:
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import sklearn as sk
from sklearn.model_selection import ShuffleSplit
from sklearn.preprocessing import OneHotEncoder

In [3]:
# load data
filename1 = "C:\\Users\\vwslz\\Desktop\\CSE512\\HW2\\data1.txt"
filename2 = "C:\\Users\\vwslz\\Desktop\\CSE512\\HW2\\data2.txt"
filename3 = "C:\\Users\\vwslz\\Desktop\\CSE512\\HW2\\data3.txt"
data1 = pd.read_csv(filename1, delimiter='\t', header=None, names = ['feature1', 'feature2', 'feature3', 'feature4', 'feature5', 'label'])
data2 = pd.read_csv(filename2, delimiter='\t', header=None, names = ['feature1', 'feature2', 'feature3', 'feature4', 'feature5', 'label'])
data3 = pd.read_csv(filename3, delimiter='\t', header=None, names = ['feature1', 'feature2', 'feature3', 'feature4', 'feature5', 'label'])

df1 = pd.DataFrame(data1)
df2 = pd.DataFrame(data2)
df3 = pd.DataFrame(data3)

data = pd.concat([data1, data2, data3], ignore_index=True)

data_X = data.iloc[:, 0:5]
data_Y = data['label']

#one hot key
onehot_encoder = OneHotEncoder(sparse=False)
onehot_encoded = onehot_encoder.fit_transform(data_Y.values.reshape(-1,1))

randomShuffle = ShuffleSplit(n_splits=10, test_size=.2, random_state=0)

max_accuracy = 0
res_W = 0
res_b = 0
res_accuracy = []

In [4]:
for train_index, test_index in randomShuffle.split(data):
    X_train = data_X.iloc[train_index, :].values
    Y_train = onehot_encoded[train_index, :]

    X_test = data_X.iloc[test_index, :].values
    Y_test = onehot_encoded[test_index, :]

    learning_rate = 1e-2

    X = tf.placeholder(tf.float32, shape=[None, 5])
    Y = tf.placeholder(tf.float32, shape=[None, 3])
    W = tf.Variable(tf.random_uniform([5, 3])*0.5, name='weight')
    b = tf.Variable(tf.random_uniform([1, 3]), name='bias')

    logits = tf.matmul(X, W) + b
    softmax = tf.exp(logits) / tf.reduce_sum(tf.exp(logits), 0)

    cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(softmax), axis = 1))
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    train = optimizer.minimize(cost)

    sess = tf.Session()
    sess.run(tf.global_variables_initializer())

    for step in range(9000):
        cost_val, _ = sess.run([cost, train], feed_dict={
            X: X_train,
            Y: Y_train})
        #if (step > 8990):
        #    print(cost_val)

    W_val, b_val, _ = sess.run([W, b, train], feed_dict={
            X: X_train,
            Y: Y_train})
            
    correct_prediction = tf.equal(tf.argmax(softmax,1), tf.argmax(Y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    accuracy_val = sess.run(accuracy, feed_dict={X: X_test, Y: Y_test})
    res_accuracy.append(accuracy_val)
    if accuracy_val > max_accuracy:
        res_W = W_val
        res_b = b_val

In [6]:
print(res_accuracy)

[0.71666664, 0.76666665, 0.75, 0.8, 0.78333336, 0.68333334, 0.8666667, 0.6666667, 0.8, 0.7]


(b) Report the best w's and b's from your classifier.

In [7]:
print(res_W)
print(res_b)

[[-0.03942344  0.09809706 -0.20068134]
 [-0.06751192  0.04120848  0.04659131]
 [-1.2031257   0.5291206  -0.4987563 ]
 [-0.36747205 -0.19375043 -0.0491154 ]
 [-0.4132177   0.1624189   0.20662223]]
[[0.0805372  0.23769689 0.20414364]]


### <font color='black'>Q2. (Multinomail Classification & Feature Reduction)</font>

In [65]:
from sklearn.utils import shuffle
from sklearn.model_selection import KFold

featurenames = ['feature1', 'feature2', 'feature3', 'feature4', 'feature5']
data_shuffle = shuffle(data)
accuracy_avg_max = 0
n_fold = 5

kf = KFold(n_splits = n_fold, shuffle = False, random_state = 2)

learning_rate = 1e-2

X = tf.placeholder(tf.float32, shape=[None, 4])
Y = tf.placeholder(tf.float32, shape=[None, 3])
W = tf.Variable(tf.random_uniform([4, 3]), name='weight')
b = tf.Variable(tf.random_uniform([1, 3]), name='bias')

In [66]:
for index in range(5):
    accuracy_sum = 0
    for train_index, test_index in kf.split(data_shuffle):
        train = data_shuffle.iloc[train_index]
        test =  data_shuffle.iloc[test_index]

        data_X_train = train.iloc[:, 0:5].drop(columns=[featurenames[index]]).values
        data_Y_train = onehot_encoder.fit_transform(train['label'].values.reshape(-1,1))
        data_X_test = test.iloc[:, 0:5].drop(columns=[featurenames[index]]).values
        data_Y_test = onehot_encoder.fit_transform(test['label'].values.reshape(-1,1))

        logits = tf.matmul(X, W) + b
        softmax = tf.exp(logits) / tf.reduce_sum(tf.exp(logits), 0)

        cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(softmax), axis = 1))
        optimizer = tf.train.GradientDescentOptimizer(learning_rate)
        train = optimizer.minimize(cost)
        
        sess = tf.Session()
        sess.run(tf.global_variables_initializer())

        for step in range(9000):
            cost_val, _ = sess.run([cost, train], feed_dict={
                X: data_X_train,
                Y: data_Y_train})

        correct_prediction = tf.equal(tf.argmax(softmax,1), tf.argmax(Y,1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        accuracy_val = sess.run(accuracy, feed_dict={X: data_X_test, Y: data_Y_test})
        accuracy_sum += accuracy_val

    accuracy_avg = accuracy_sum / n_fold
    print("The accuracy of model to drop feature " + str(index + 1) + ": " + str(accuracy_avg))

The accuracy of model to drop feature 1: 0.7133333444595337
The accuracy of model to drop feature 2: 0.6933333277702332
The accuracy of model to drop feature 3: 0.526666671037674
The accuracy of model to drop feature 4: 0.65
The accuracy of model to drop feature 5: 0.7666666626930236


According to the above accuracies, the mode dropping feature 5 has the highest accuracy. In other words, feature 5 has the least influence on the results. So I would like to select the other features for my classifier.

In [67]:
X = tf.placeholder(tf.float32, shape=[None, 4])
Y = tf.placeholder(tf.float32, shape=[None, 3])
W = tf.Variable(tf.random_uniform([4, 3])*0.5, name='weight')
b = tf.Variable(tf.random_uniform([1, 3]), name='bias')

learning_rate = 1e-3

max_accuracy = 0
res_W = 0
res_b = 0
res_accuracy = []

data_X = data.iloc[:, 0:5].drop(columns=['feature5'])
data_Y = data['label']

sess = tf.Session()

In [68]:
for train_index, test_index in randomShuffle.split(data):  
    X_train = data_X.iloc[train_index, :].values
    Y_train = onehot_encoded[train_index, :]

    X_test = data_X.iloc[test_index, :].values
    Y_test = onehot_encoded[test_index, :]
    
    logits = tf.matmul(X, W) + b
    softmax = tf.exp(logits) / tf.reduce_sum(tf.exp(logits), 0)

    cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(softmax), axis = 1))
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    train = optimizer.minimize(cost)

    sess = tf.Session()
    sess.run(tf.global_variables_initializer())

    for step in range(9000):
        cost_val, _ = sess.run([cost, train], feed_dict={
            X: X_train,
            Y: Y_train})

    W_val, b_val, _ = sess.run([W, b, train], feed_dict={
            X: X_train,
            Y: Y_train})
            
    correct_prediction = tf.equal(tf.argmax(softmax,1), tf.argmax(Y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    accuracy_val = sess.run(accuracy, feed_dict={X: X_test, Y: Y_test})
    res_accuracy.append(accuracy_val)
    if accuracy_val > max_accuracy:
        res_W = W_val
        res_b = b_val

(a) Randomly shuffle each dataset and use 80% of data for training and 20% for testing. And repeat 10 times and report the average accuracy for each class.

In [70]:
print (res_accuracy)

[0.71666664, 0.76666665, 0.8, 0.81666666, 0.78333336, 0.7, 0.8666667, 0.78333336, 0.78333336, 0.73333335]


(b) Report the best w's and b's from your classifier.

In [71]:
print(res_W)
print(res_b)

[[-0.09435022  0.11295945 -0.16324764]
 [-0.04381175  0.02561875  0.0136826 ]
 [-0.37269166  0.5630736  -0.1602366 ]
 [-0.72112477 -0.04416402  0.12874234]]
[[0.01960063 0.07007885 0.9649017 ]]


(c) Compare the accuracy with Q1.(a). Was it better or worse by how much? Explain your reason for selecting 4 features.

Although the best accuracy is the same, the average accuracy is improved. In Q1(a), the average accuracy is 0.753333339 while in Q2(a) the average accuracy is 0.775000008. Decrease the number features can help the model decrease the influence from noises. In other words, it helps to reduce variance.

### <font color='black'>Q3. (SVM)</font>

In [26]:
%matplotlib inline

from sklearn.svm import SVC
from sklearn.utils import shuffle
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import sklearn as sk
import seaborn as sns; sns.set(font_scale=1.2)

In [60]:
filename1 = "C:\\Users\\vwslz\\Desktop\\CSE512\\HW2\\data1.txt"
filename2 = "C:\\Users\\vwslz\\Desktop\\CSE512\\HW2\\data2.txt"
filename3 = "C:\\Users\\vwslz\\Desktop\\CSE512\\HW2\\data3.txt"
data1 = pd.read_csv(filename1, delimiter='\t', header=None, names = ['feature1', 'feature2', 'feature3', 'feature4', 'feature5', 'label'])
data2 = pd.read_csv(filename2, delimiter='\t', header=None, names = ['feature1', 'feature2', 'feature3', 'feature4', 'feature5', 'label'])
data3 = pd.read_csv(filename3, delimiter='\t', header=None, names = ['feature1', 'feature2', 'feature3', 'feature4', 'feature5', 'label'])

df1 = pd.DataFrame(data1)
df2 = pd.DataFrame(data2)
df3 = pd.DataFrame(data3)

data = pd.concat([data1, data2, data3], ignore_index=True)
data_shuffle = shuffle(data)

onehot_encoder = OneHotEncoder(sparse=False)

data_X = data.iloc[:, 0:5].values
#data_Y = onehot_encoder.fit_transform(data['label'].values.reshape(-1,1))
data_Y = data['label'].values

X_train_shuffle, X_test_shuffle, Y_train_shuffle, Y_test_shuffle = train_test_split(
    data_X, data_Y, test_size=0)

In [92]:

from sklearn.model_selection import GridSearchCV

tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100, 1000],
                     'C': [1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100, 1000]}]

score='accuracy'

print("# Tuning hyper-parameters for %s" % score)
print()

clf = GridSearchCV(SVC(), tuned_parameters, cv=5, scoring=score)
clf.fit(X_train_shuffle, Y_train_shuffle)

print("Best parameters set found on development set:")
print(clf.best_params_)
#print()
#print("Grid scores on development set:")
#means = clf.cv_results_['mean_test_score']
#stds = clf.cv_results_['std_test_score']
#for mean, std, params in zip(means, stds, clf.cv_results_['params']):
#    print("%0.3f (+/-%0.03f) for %r"
#          % (mean, std * 2, params))

# Tuning hyper-parameters for accuracy

Best parameters set found on development set:
{'C': 100, 'gamma': 0.01, 'kernel': 'rbf'}


In [94]:
from sklearn.metrics import accuracy_score

res_accuracy = []

model = svm.SVC(kernel='rbf', C=100, gamma=0.01)
model.fit(X_train, Y_train)

SVC(C=100, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.01, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [95]:
for i in range(10):
    X_train, X_test, Y_train, Y_test = train_test_split(
    data_X, data_Y, test_size=0.2)
    
    accuracy = accuracy_score(Y_test, model.predict(X_test))
    res_accuracy.append(accuracy)

In [96]:
print(res_accuracy)

[0.8666666666666667, 0.8666666666666667, 0.9166666666666666, 0.8833333333333333, 0.9, 0.9, 0.8166666666666667, 0.85, 0.9333333333333333, 0.8333333333333334]
